In [10]:
import easyocr
import cv2
import numpy as np
from skimage.morphology import remove_small_objects
from ultralytics import YOLO
class natioanl_Id_reader:
    def __init__(self) -> None:
        self.card_segmentor=YOLO("card_finder_seg.pt")
        self.card_dividor=YOLO("card_divider_model.pt")
        self.reader = easyocr.Reader(['ar'], gpu=True)
        pass
    def cropLocations(self, frame, bbox: list):
            '''
            This function is for cropping the image of the car plate
            args:
                frame: the frame to be cropped
                bbox: the bounding box of the car plate
            return:
                cropped: the cropped image of the car plate
            '''
            x1, y1, w, h = bbox
            cropped = frame[int(y1):int(h), int(x1):int(w)]
            cropped = cropped[int(0):int(cropped.shape[0]),
                            int(0):int(cropped.shape[1])]
            return cropped
    def cropImage(self, frame, bbox: list):
            '''
            This function is for cropping the image of the car plate
            args:
                frame: the frame to be cropped
                bbox: the bounding box of the car plate
            return:
                cropped: the cropped image of the car plate
            '''
            x1, y1, w, h = bbox[0][0:4]
            cropped = frame[int(y1):int(h), int(x1):int(w)]
            cropped = cropped[int(0):int(cropped.shape[0]),
                            int(0):int(cropped.shape[1])]
            return cropped
    def Get_the_Id_bbox(self,frame):
        '''
        This function is for getting the bounding box of the ID
        args:
            frame: the frame to be cropped
        return:
            cropped: the cropped image of the national ID
        '''
        card=self.card_segmentor.predict(frame,conf=0.8,verbose=False)
        if len(card[0].boxes.data.tolist()) != 0:
            matrix = card[0].boxes.data.tolist()
            return matrix[0:3]       
    def Get_the_Id_content(self,frame):
        '''
        This function is for getting the content of the ID
        args:
            frame: the frame to be cropped
        return:
            cropped: the cropped image of given slot in national ID
        '''
        name=self.card_dividor.names
        card=self.card_dividor.predict(frame,verbose=False)
        location_of_slots={}
        if len(card[0].boxes.data.tolist()) != 0:
            matrix = card[0].boxes.data.tolist()
            for i in matrix:
                location_of_slots[name[int(i[5])]]=i[0:4]
            return location_of_slots 
    def FilterStack(self,frame):
        '''
        This function is for filtering the image of the ID
        filter stack:
            1- convert to gray scale
            2- apply gaussian blur
            3- apply adaptive threshold
            4- apply median blur
            5- apply erode
        args:
            frame: the frame to be filtered
        return:
            frame: the filtered image of the national ID
        '''
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.GaussianBlur(frame, (3, 3), 3)
        # frame=cv2.equalizeHist(frame)
        frame = cv2.adaptiveThreshold(
            frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 7, 2)
        frame = cv2.medianBlur(frame, 3)
        kernel = np.ones((2, 2), np.uint8)
        # frame=cv2.inRange(frame,0,32)
        return frame
    def FilterStack2Id(self,frame):
        '''
        This function is for filtering the image of the ID
        filter stack:
            1- convert to gray scale
            2- apply equalize histogram
            3- apply inRange
        args:
            frame: the frame to be filtered
        return:
            frame: the filtered image of the national ID
        '''
        frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        frame=cv2.equalizeHist(frame)
        frame=cv2.inRange(frame,0,20)
        return frame
    def extract_information(self,frame):
        '''
        This function is for extracting the information from the national ID
        args:
            frame: the frame to be filtered
        return:
            name: the name of the person in the ID
            city: the city of the person in the ID
            gov: the governorate of the person in the ID
            id: the id number of the person in the ID
        '''
        im_original=frame
        bbox=self.Get_the_Id_bbox(im_original)
        croped=self.cropImage(im_original,bbox)
        location=self.Get_the_Id_content(croped)

        #divide the id into slots
        firstName_frame=self.cropLocations(croped,location["firstName"])
        name_frame=self.cropLocations(croped,location["name"])
        city_frame=self.cropLocations(croped,location["city"])
        gov_frame=self.cropLocations(croped,location["gov"])
        id_frame=self.cropLocations(croped,location["idNo"])
        #end of dividing the id into slots

        name_frame=cv2.resize(name_frame,(358,53))
        firstName_frame=cv2.resize(firstName_frame,(106,49))
        city_frame=cv2.resize(city_frame,(104,47))
        gov_frame=cv2.resize(gov_frame,(196,90))
        id_frame=cv2.resize(id_frame,(451,53))

        #filtering the frames
        name_frame=self.FilterStack(name_frame)
        firstName_frame=self.FilterStack(firstName_frame)
        city_frame=self.FilterStack(city_frame)
        gov_frame=self.FilterStack(gov_frame)
        id_frame=self.FilterStack2Id(id_frame)
        #end of filtering the frames

        #reading the text
        name=self.reader.readtext(name_frame,detail=0,paragraph=True)
        firstName=self.reader.readtext(firstName_frame,detail=0,paragraph=True)
        city=self.reader.readtext(city_frame,detail=0,paragraph=True)
        gov=self.reader.readtext(gov_frame,detail=0,paragraph=True)
        id=self.reader.readtext(id_frame,
                                detail=0,
                                paragraph=True,
                                rotation_info =[i for i in range(0,270,10)],
                                text_threshold = 0.1,
                                low_text = 0.1,link_threshold=0.1,canvas_size=10000)
        return firstName[0]+" "+name[0],city,gov,id



In [15]:
import pandas as pd
# read the image of the national ID
im=cv2.imread("id_card_3.jpg")
reader=natioanl_Id_reader()
name,city,gov,id=reader.extract_information(im)
df=pd.DataFrame({"Name":name,"City":city,"Gov":gov,"Id Number":id})
df.to_csv(f"{name}_Id_content.csv",index=False)
df.head()

,Name,City,Gov,Id Number
0,ابراهيم عبدالغفار ابراهيم عبدالغفار,المطزبة,القاهره,١٥ ٠٠٧ ٠١ ٢١ ٠٦ ٩٩ ٢


In [16]:
im=cv2.imread("id_card_1.jpg")
reader=natioanl_Id_reader()
name,city,gov,id=reader.extract_information(im)
df=pd.DataFrame({"Name":name,"City":city,"Gov":gov,"Id Number":id})
df.to_csv(f"{name}_Id_content.csv",index=False)
df.head()

,Name,City,Gov,Id Number
0,محمول عصام عبدالعزيز قطب محمد,الزقازبن تان,الشرقية,٩٥ ٠٠٥ ١٣ ٠٩ ٠٩ ٨٩ ٢
